# Setup 

## Installing modelscan

In [1]:
!pip install modelscan

  Obtaining dependency information for modelscan from https://files.pythonhosted.org/packages/bf/6e/dda8f775b53c307bd59042a7d20143b3f0d068e9b9c6776cf07a28ce208d/modelscan-0.1.1-py3-none-any.whl.metadata
  Using cached modelscan-0.1.1-py3-none-any.whl.metadata (5.1 kB)
  Obtaining dependency information for click<9.0.0,>=8.1.3 from https://files.pythonhosted.org/packages/1a/70/e63223f8116931d365993d4a6b7ef653a4d920b41d03de7c59499962821f/click-8.1.6-py3-none-any.whl.metadata
  Using cached click-8.1.6-py3-none-any.whl.metadata (3.0 kB)
  Using cached numpy-1.24.0-cp39-cp39-macosx_11_0_arm64.whl (13.9 MB)
  Obtaining dependency information for rich<14.0.0,>=13.4.2 from https://files.pythonhosted.org/packages/8d/5f/21a93b2ec205f4b79853ff6e838e3c99064d5dbe85ec6b05967506f14af0/rich-13.5.2-py3-none-any.whl.metadata
  Using cached rich-13.5.2-py3-none-any.whl.metadata (18 kB)
  Obtaining dependency information for markdown-it-py>=2.2.0 from https://files.pythonhosted.org/packages/42/d7/1ec15b4

In [2]:
!pip install -q torch==2.0.1
!pip install -q transformers==4.31.0
!pip install -q scipy==1.11.1

In [3]:
import torch
import os 
from utils.pytorch_sentiment_model import download_model, predict_sentiment
from utils.pickle_codeinjection import PickleInject, get_payload
%env TOKENIZERS_PARALLELISM=false


/Users/mehrinkiani/mambaforge/envs/notebooks-pt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


env: TOKENIZERS_PARALLELISM=false


# Saving Model


The BERT based sentiment analysis PyTorch model used in the notebook can be found at https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment. The safe model is saved at `./PyTorchModels/safe_model.pt`

In [4]:
# Save a model for sentiment analysis
model_directory = "PyTorchModels"
if not os.path.isdir(model_directory):
    os.mkdir(model_directory)

safe_model_path = os.path.join(model_directory, "safe_model.pt")

sentiment_model = download_model(safe_model_path)

# Safe Model Prediction

In [5]:
sentiment = predict_sentiment("Stock market was bearish today", torch.load(safe_model_path))

The overall sentiment is: negative with a score of: 85.9%


# Scan the Safe Model

The scan results include information on the files scanned, and any issues if found. For the safe model scanned, modelscan finds no model serialization attacks, as expected.

In [6]:
!modelscan --path PyTorchModels/safe_model.pt

Scanning /Users/mehrinkiani/Documents/modelscan/notebooks/PyTorchModels/safe_model.pt:safe_model/data.pkl using pickle model scan

--- Summary ---

 No issues found! 🎉


# Model Serialization Attack

Here malicious code is injected in the safe model to read aws secret keys. The unsafe model is saved at `./PyTorchModels/unsafe_model.pt`

In [7]:
command = "system"
malicious_code = """cat ~/.aws/secrets
    """

unsafe_model_path = os.path.join(model_directory, "unsafe_model.pt")


payload = get_payload(command, malicious_code)
torch.save(
    torch.load(safe_model_path),
    f=unsafe_model_path,
    pickle_module=PickleInject([payload]),
)

# Unsafe Model Prediction

The malicious code injected in the unsafe model gets executed when it is loaded. The aws secret keys are displayed. 

Also, the unsafe model predicts the sentiments just as well as safe model i.e., the code injection attack will not impact the model performance. The unaffected performance of unsafe models makes the ML models an effective attack vector. 


In [8]:
sentiment = predict_sentiment("Stock market was bearish today", torch.load(unsafe_model_path))

aws_access_key_id=<access_key_id>
aws_secret_access_key=<aws_secret_key>
The overall sentiment is: negative with a score of: 85.9%


# Scanning Unsafe Model

The scan results include information on the files scanned, and any issues if found. In this case, a critical severity level issue is found in the unsafe model scanned. 

modelscan also outlines the found operator(s) and module(s) deemed unsafe. 

In [9]:
!modelscan --path  ./PyTorchModels/unsafe_model.pt

Scanning /Users/mehrinkiani/Documents/modelscan/notebooks/PyTorchModels/unsafe_model.pt:unsafe_model/data.pkl using pickle model scan

--- Summary ---

Total Issues: 1

Total Issues By Severity:

    - LOW: 0
    - MEDIUM: 0
    - HIGH: 0
    - CRITICAL: 1

--- Issues by Severity ---

--- CRITICAL ---

Unsafe operator found:
  - Severity: CRITICAL
  - Description: Use of unsafe operator 'system' from module 'posix'
  - Source: /Users/mehrinkiani/Documents/modelscan/notebooks/PyTorchModels/unsafe_model.pt:unsafe_model/data.pkl
